In [1]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=360bb569cc62ee81060503b9107d43cd6d4da77e275e4e8e95362d826eb0159d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 120899 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u392-ga-1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u392-ga-1~22.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u392-ga-1~22.0

##Drive on

In [2]:
import pyspark
import numpy as np
import pandas as pd
import os

from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, broadcast, udf
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import FloatType
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [81]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "gdrive/MyDrive/Final Data/"
PATH = "gdrive/MyDrive/추천시스템/정리/data/"
os.listdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['1210 all_pp.csv',
 '1210 CosSim_All.csv',
 '1210 CosSim_Title.csv',
 '1210 CosSim_Genres.csv',
 '1210 CosSim_Tag.csv',
 '1210 CosSim_Story.csv']

In [ ]:
cosSim_All    = pd.read_csv(path + '1210 CosSim_All.csv')
cosSim_Genres = pd.read_csv(path + '1210 CosSim_Genres.csv')
cosSim_Title  = pd.read_csv(path + '1210 CosSim_Title.csv')
cosSim_Tag    = pd.read_csv(path + '1210 CosSim_Tag.csv')
cosSim_Story  = pd.read_csv(path + '1210 CosSim_Story.csv')

In [ ]:
EucSim_All    =  pd.read_csv(path + '1210 EucSim_All.csv')
EucSim_Genres = pd.read_csv(path + '1210 EucSim_Genres.csv')
EucSim_Title  = pd.read_csv(path + '1210 EucSim_Title.csv')
EucSim_Tag    = pd.read_csv(path + '1210 EucSim_Tag.csv')
EucSim_Story  = pd.read_csv(path + '1210 EucSim_Story.csv')

In [5]:
k = pd.read_csv(path + "1210 all_pp.csv")
movie = pd.read_csv(PATH + "movies_refined.csv")
rating = pd.read_csv(PATH + 'ratings_refined.csv')

##Merging and preprocessing files

In [6]:
new_data = pd.merge(movie, k, on='movieId', how='right')

In [7]:
new_df = pd.merge(movie, k, on='movieId', how='left')

In [8]:
new_ratings = pd.merge(rating, k, on='movieId', how='left')

In [9]:
title_list = new_data['title_x'].tolist()

In [10]:
movie_id_list = new_data['movieId'].tolist()

###Set Euclidean matrix index

In [ ]:
EucSim_All_id    = EucSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_All.columns, movie_id_list)))
EucSim_Genres_id = EucSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Genres.columns, movie_id_list)))
EucSim_Title_id  = EucSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Title.columns, movie_id_list)))
EucSim_Tag_id    = EucSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Tag.columns, movie_id_list)))
EucSim_Story_id  = EucSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Story.columns, movie_id_list)))

In [ ]:
EucSim_All    = EucSim_All.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_All.columns, title_list)))
EucSim_Genres = EucSim_Genres.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Genres.columns, title_list)))
EucSim_Title  = EucSim_Title.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Title.columns, title_list)))
EucSim_Tag    = EucSim_Tag.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Tag.columns, title_list)))
EucSim_Story  = EucSim_Story.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Story.columns, title_list)))

ValueError: ignored

###Set cosine matrix index

In [ ]:
cosSim_All_id    = cosSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_All.columns, movie_id_list)))
cosSim_Genres_id = cosSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Genres.columns, movie_id_list)))
cosSim_Title_id  = cosSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Title.columns, movie_id_list)))
cosSim_Tag_id    = cosSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Tag.columns, movie_id_list)))
cosSim_Story_id  = cosSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Story.columns, movie_id_list)))

In [ ]:
cosSim_All    = cosSim_All.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_All.columns, title_list)))
cosSim_Genres = cosSim_Genres.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Genres.columns, title_list)))
cosSim_Title  = cosSim_Title.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Title.columns, title_list)))
cosSim_Tag    = cosSim_Tag.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Tag.columns, title_list)))
cosSim_Story  = cosSim_Story.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Story.columns, title_list)))

###Rating pre-processing

In [12]:
new_ratings_refined = rating[rating['movieId'].isin(movie_id_list)]

In [13]:
new_ratings_refined = pd.merge(new_ratings_refined, movie, on='movieId', how='left')

In [14]:
new_ratings_refined = new_ratings_refined[['userId', 'movieId', 'rating', 'title']]

In [15]:
new_ratings_refined

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,1,3,4.0,Grumpier Old Men
2,1,6,4.0,Heat
3,1,47,5.0,Seven (a.k.a. Se7en)
4,1,50,5.0,"Usual Suspects, The"
...,...,...,...,...
100694,610,166534,4.0,Split
100695,610,168248,5.0,John Wick: Chapter Two
100696,610,168250,5.0,Get Out
100697,610,168252,5.0,Logan


##Modeling

### 1. python

In [29]:
from tqdm.notebook import tqdm
import numpy as np
def modeling(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    titles = title_list
    all_users = sorted(data['userId'].unique())
    n_titles = len(titles)

    for user in tqdm(all_users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_movies = data.loc[idx, 'movieId'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movies]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1


        # 평점 예측
        watched_movies_y = data.loc[idx, 'rating']
        watched_movies_y = np.array(watched_movies_y.tolist()).reshape(-1, 1)

        pred_y = np.matmul(sub_sim_mat, watched_movies_y).flatten() / sim_N

        users_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(users_list, titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)

    return df_pred_all

### 2. Spark

In [24]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np

def calculate_predictions(sub_sim_mat, watched_movies_y, sim_N):
    # Initialize SparkContext
    sc = pyspark.SparkContext()
    spark = SparkSession.builder.getOrCreate()

    # sc = SparkContext("local", "Spark Calculation")
    # spark = SparkSession(sc)

    # Create RDDs for sub_sim_mat, watched_movies_y, and sim_N
    sub_sim_mat_rdd = sc.parallelize(sub_sim_mat)
    watched_movies_y_rdd = sc.parallelize(watched_movies_y)
    sim_N_bc = sc.broadcast(sim_N)

    # Calculate pred_y using Spark transformations and actions
    def calculate_pred_y(sub_sim_mat_row):
        return np.dot(sub_sim_mat_row, watched_movies_y) / sim_N_bc.value

    pred_y_rdd = sub_sim_mat_rdd.map(calculate_pred_y)
    pred_y = pred_y_rdd.collect()

    # Flatten the result if necessary
    pred_y_flat = np.array(pred_y).flatten()

    # Stop SparkContext
    sc.stop()

    return pred_y_flat


In [25]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

def modeling_spark(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    titles = title_list
    all_users = sorted(data['userId'].unique())
    n_titles = len(titles)

    for user in tqdm(all_users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_movies = data.loc[idx, 'movieId'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movies]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1

        # 평점 예측
        watched_movies_y = data.loc[idx, 'rating']
        watched_movies_y = np.array(watched_movies_y.tolist()).reshape(-1, 1)

        # pred_y 계산 (PySpark)
        pred_y_spark = calculate_predictions(sub_sim_mat, watched_movies_y, sim_N)

        # pred_y를 NumPy 배열로 변환하고 계산
        pred_y = np.array(pred_y_spark)
        users_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(users_list, titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)

    return df_pred_all

##Rating prediction

###Cosine_simility

####all

In [27]:
df_pred_cos_all_sp = modeling_spark(cosSim_All_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

ConnectionRefusedError: ignored

In [30]:
df_pred_cos_all= modeling(cosSim_All_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Story

In [ ]:
df_pred_cos_story=modeling(cosSim_Story_id,new_ratings_refined)

NameError: ignored

####Genre

In [ ]:
df_pred_cos_genre=modeling(cosSim_Genres_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Title

In [ ]:
df_pred_cos_title=modeling(cosSim_Title_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Tag

In [ ]:
df_pred_cos_tag=modeling(cosSim_Tag_id,new_ratings_refined)

NameError: ignored

####Story * Title * Genre * Tag


###Euclidean Similarity

####All

In [ ]:
df_pred_euc_all=modeling(EucSim_All_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####Story

In [ ]:
df_pred_euc_story=modeling(EucSim_Story_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####Genre

In [ ]:
df_pred_euc_genre=modeling(EucSim_Genres_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####Title

In [ ]:
df_pred_euc_title=modeling(EucSim_Title_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####Tag

In [ ]:
df_pred_euc_tag=modeling(EucSim_Tag_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####Story * Title * Genre * Tag

## Evaluation

In [34]:
from sklearn.model_selection import train_test_split

_, test_data = train_test_split(new_ratings_refined[['userId', 'title', 'rating']],
                test_size=20000, random_state=1234, stratify=new_ratings_refined['userId']) # 2만개 샘플링
test_data

,userId,title,rating
48238,313,Star Wars: Episode VI - Return of the Jedi,3.0
36923,249,Push,3.5
36015,246,Arizona Dream,4.0
64042,414,Finding Nemo,4.5
85364,555,Batman Returns,3.0
...,...,...,...
556,5,Pinocchio,5.0
10198,66,Beverly Hills Cop,3.0
53959,356,"Bourne Supremacy, The",4.0
35434,239,28 Days Later,5.0


### MAE, MSE, RMSE

In [42]:
def calculate_errors(test_data):
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
    mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
    rmse = np.sqrt(mse)

    calculate_errors = []
    print(f"MAE  : {str(round(mae, 2))}")
    print(f"MSE  : {str(round(mse, 2))}")
    print(f"RMSE : {str(round(rmse, 2))}")

    calculate_errors.append(mae)
    calculate_errors.append(mse)
    calculate_errors.append(rmse)

    return calculate_errors

 ### Precision, Recall

In [59]:
import numpy as np

def get_precision_recall_f1(test_data):
    threshold = 3
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    # True positives and negatives
    trues = 1 * (true_y >= threshold)
    pred_trues = 1 * (pred_y >= threshold)

    # Calculate True Positives
    true_positive = sum(trues & pred_trues)

    # Precision Calculation
    predicted_positives = sum(pred_trues)
    precision = true_positive / predicted_positives if predicted_positives else 0

    # Recall Calculation
    false_negatives = sum(trues & ~pred_trues)
    recall = true_positive / (true_positive + false_negatives) if (true_positive + false_negatives) else 0

    # F1-Score Calculation
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    print(f"Precision: {round(precision, 2)}")
    print(f"Recall: {round(recall, 2)}")
    print(f"F1-Score: {round(f1_score, 2)}")

    result = []
    result.append(precision)
    result.append(recall)
    result.append(f1_score)

    return result


### NDCG

In [76]:
def dcg_at_k(relevances, k=10):
    relevances = np.asfarray(relevances)[:k]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)

    return np.sum(relevances / discounts)

def ndcg_at_k(test_data, k=10):
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    idcg = dcg_at_k(sorted(true_y, reverse=True), k)
    dcg = dcg_at_k(pred_y, k)
    ndcg = dcg / idcg if idcg > 0 else 0

    # 결과 출력
    print(f"NDCG Score : {round(ndcg, 2)}")

    return ndcg

#### Cosine_simility

#####All

In [37]:
test_data_cos_all = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.398480
1,249,Push,3.5,3.709894
2,246,Arizona Dream,4.0,4.213896
3,414,Finding Nemo,4.5,3.350313
4,555,Batman Returns,3.0,3.472098
...,...,...,...,...
21471,5,Pinocchio,5.0,3.427091
21472,66,Beverly Hills Cop,3.0,3.986162
21473,356,"Bourne Supremacy, The",4.0,3.963656
21474,239,28 Days Later,5.0,4.015459


In [79]:
print("Cosine_simility All \n")
cos_all = calculate_errors(test_data_cos_all)
cos_all.append(get_precision_recall_f1(test_data_cos_all))
cos_all.append(ndcg_at_k(test_data_cos_all))

Cosine_simility All 

MAE  : 0.72
MSE  : 0.85
RMSE : 0.92
Precision: 0.86
Recall: 0.9
F1-Score: 0.88
NDCG Score : 0.73


##### Story

In [ ]:
test_data_cos_story = pd.merge(test_data, df_pred_cos_story, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_story

In [80]:
print("Cosine_simility Story \n")
cos_story = calculate_errors(test_data_cos_story)
cos_story.append(get_precision_recall_f1(test_data_cos_story))
cos_story.append(ndcg_at_k(test_data_cos_story))

Cosine_simility Story 



NameError: ignored

##### Genre

In [ ]:
test_data_cos_genre = pd.merge(test_data, df_pred_cos_genre, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_genre

In [ ]:
print("Cosine_simility Genre \n")
cos_genre = calculate_errors(test_data_cos_genre)
cos_genre.append(get_precision_recall_f1(test_data_cos_genre))
cos_genre.append(ndcg_at_k(test_data_cos_genre))

##### Title

In [ ]:
test_data_cos_title= pd.merge(test_data, df_pred_cos_title, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_title

In [ ]:
print("Cosine_simility Title \n")
cos_title = calculate_errors(test_data_cos_title)
cos_title = get_precision_recall_f1(test_data_cos_title)
cos_title.append(ndcg_at_k(test_data_cos_title))

##### Tag

In [ ]:
test_data_cos_tag = pd.merge(test_data, df_pred_cos_tag, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_tag

In [ ]:
print("Cosine_simility Tag \n")
cos_tag = calculate_errors(test_data_cos_tag)
cos_tag.append(get_precision_recall_f1(test_data_cos_tag))
cos_tag.append(ndcg_at_k(test_data_cos_tag))

#### Euclidean Similarity

##### All

In [ ]:
test_data_euc_all= pd.merge(test_data, df_pred_euc_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_all

In [ ]:
print("Euclidean Similarity All \n")
euc_all = calculate_errors(test_data_euc_all)
euc_all.append(get_precision_recall_f1(test_data_euc_all))
euc_all.append(ndcg_at_k(test_data_euc_all))

##### Story

In [ ]:
test_data_euc_story = pd.merge(test_data, df_pred_euc_story, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_story

In [ ]:
print("Euclidean Similarity stroy \n")
euc_story = calculate_errors(test_data_euc_story)
euc_story.append(get_precision_recall_f1(test_data_euc_story))
euc_story.append(ndcg_at_k(test_data_euc_story))

##### Genre

In [ ]:
test_data_euc_genre = pd.merge(test_data, df_pred_euc_genre, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_genre

In [ ]:
print("Euclidean Similarity genre \n")
euc_genere = calculate_errors(test_data_euc_genre)
euc_genere.append(get_precision_recall_f1(test_data_euc_genre))
euc_genere.append(ndcg_at_k(test_data_euc_genre))

##### Title

In [ ]:
test_data_euc_title = pd.merge(test_data, df_pred_euc_title, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_title

In [ ]:
print("Euclidean Similarity title \n")
euc_title = calculate_errors(test_data_euc_title)
euc_title.append(get_precision_recall_f1(test_data_euc_title))
euc_title.append(ndcg_at_k(test_data_euc_title))

##### Tag

In [ ]:
test_data_euc_tag = pd.merge(test_data, df_pred_euc_tag, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_tag

In [ ]:
print("Euclidean Similarity tag \n")
euc_tag = calculate_errors(test_data_euc_tag)
euc_tag.append(get_precision_recall_f1(test_data_euc_tag))
euc_tag.append(ndcg_at_k(test_data_euc_tag))

##Recomendation

In [170]:
user_id = 1

###Cosine_simility

####All

In [168]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_all[df_pred_cos_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Story

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_story[df_pred_cos_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Genre

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_genre[df_pred_cos_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Title

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_title[df_pred_cos_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Tag

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_tag[df_pred_cos_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

###Euclidean Similarity

####All

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_all[df_pred_euc_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Story

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_story[df_pred_euc_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Genre

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_genre[df_pred_euc_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Title

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_title[df_pred_euc_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####Tag

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_tag[df_pred_euc_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()